In [ ]:
import sys
sys.path.append("../")

In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
from technicals.indicators import rsi
from technicals.patterns import apply_patterns
from simulation.guru_tester import GuruTester

In [ ]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0


def apply_signal(df):
    df['signal'] = np.where(df['engulfing'] == True,
                                 np.where((df['direction'] == BUY) & (df['mid_l'] > df['ema_200']) & (df['RSI_14'] > RSI_LIMIT), BUY,
                                          np.where((df['direction'] == SELL) & (df['mid_h'] < df['ema_200']) & (df['RSI_14'] < RSI_LIMIT), SELL, NONE)),
                                            NONE)
    return df['signal']
  

In [ ]:
def run_pair(pair):
    df_an = pd.read_pickle(f"../data/{pair}_H1.pkl")
    df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
    
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    
    df_an = rsi(df_an)
    df_an = apply_patterns(df_an)
    df_an['ema_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()
    
    our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'bid_o', 'bid_h', 'bid_l', 'bid_c', 
            'ask_o', 'ask_h', 'ask_l', 'ask_c',
            'engulfing', 'direction', 'ema_200', 'RSI_14' ]
    
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)
    df_slim.reset_index(drop=True, inplace=True)
    
    gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
        use_spread=True
    )
    
    gt.run_test()
    return gt.df_results
    

In [ ]:
results = []
for pair in ['USD_CHF']:
    results.append(dict(pair=pair, result=run_pair(pair)))

In [ ]:
for result in results:
    print(result['pair'], result['result'].result.sum())